In [87]:
import pickle
import pandas as pd
import numpy as np
from copy import deepcopy
import math
from sklearn.model_selection import train_test_split

activities_to_classify = {
        'Get/replace items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open/close a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

FPS = 30
ACTION_LENGTH = 5

def augmentation(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = FPS * ACTION_LENGTH 

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/FPS
            new_row['stop_timestamp'] = new_stop/FPS
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def create_annotations_file(timestamp_file, annotations_file, type='train'):

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_frame'] = ((timestamps['start'] - start_timestamp) * FPS).astype(int)
    timestamps['stop_frame'] = ((timestamps['stop'] - start_timestamp) * FPS).astype(int)
    
    annotations = pd.read_pickle(annotations_file)
    annotations = annotations[annotations['file'] == 'S04_1.pkl']
    annotations = annotations.drop(['labels'], axis=1)
    
    complete_labels = pd.merge(timestamps, annotations, on='index', how='inner')
    
    complete_labels['uid'] = complete_labels['index']
    complete_labels['participant_id'] = 'S04'
    complete_labels['video_id'] = 'S04_1'
    complete_labels['verb'] = complete_labels['description_x']
    complete_labels['narration'] = complete_labels['description_x']
    complete_labels['verb_class'] = complete_labels['verb'].map(activities_to_classify)

    complete_labels = complete_labels[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    
    complete_labels['type'] = type
    
    return complete_labels
    
    
def augment_annotations(train_annotations, test_annotations):
    
    combined_df = pd.concat([train_annotations, test_annotations], ignore_index=True)
    augmented_df = augmentation(combined_df)
    
    augmented_df = augmented_df.sample(frac=1)

    augmented_df.reset_index(inplace=True)
    augmented_df['uid'] = augmented_df.index
    
    train_df_final = augmented_df[augmented_df['type'] == 'train']
    test_df_final = augmented_df[augmented_df['type'] == 'test']
    
    train_df_final = train_df_final.drop(['type'], axis=1)
    test_df_final = test_df_final.drop(['type'], axis=1)

    train_df_final.to_pickle(f"action_net_annotations_training/S04_train.pkl")    
    test_df_final.to_pickle(f"action_net_annotations_training/S04_test.pkl")   

def take_S04_annotations_RGB(timestamps, emg_data, type='train'):
    
    calibration_val = pd.read_pickle(timestamps)['start'].iloc[0]

    emg_data = pd.read_pickle(emg_data)
    emg_data = emg_data[emg_data['file'] == 'S04_1.pkl']
    emg_data = emg_data.rename(columns={'file': 'video_id', 'description': 'narration', 'description_class': 'verb_class'})
    emg_data['participant_id'] = 'S04'
    emg_data['video_id'] = 'S04_1'
    emg_data['start_frame'] = ((emg_data['start'] - calibration_val) * FPS).astype(int)
    emg_data['stop_frame'] = ((emg_data['stop'] - calibration_val) * FPS).astype(int)
    emg_data['verb'] = emg_data['narration']
    emg_data = emg_data.drop(['emg_data'], axis = 1)
    emg_data = emg_data[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    emg_data = emg_data.sample(frac=1).reset_index(drop=True)
    
    emg_data['type'] = type
    
    return emg_data

def create_reduced_annotations(train_annotations, test_annotations):
    
    combined_df = pd.concat([train_annotations, test_annotations], ignore_index=True)
    
    combined_df = combined_df.sample(frac=1)

    combined_df.reset_index(inplace=True)
    combined_df['uid'] = combined_df.index
    
    train_df_final = combined_df[combined_df['type'] == 'train']
    test_df_final = combined_df[combined_df['type'] == 'test']
    
    train_df_final = train_df_final.drop(['type'], axis=1)
    test_df_final = test_df_final.drop(['type'], axis=1)

    train_df_final.to_pickle(f"action_net_annotations_training/S04_train.pkl")    
    test_df_final.to_pickle(f"action_net_annotations_training/S04_test.pkl")   
    
def create_multimodal_annotations(full_data, split):
    full_data = pd.read_pickle(full_data)
    full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'description_class': 'verb_class'})
    full_data['participant_id'] = 'S04'
    full_data['video_id'] = 'S04_1'
    
    final_annotations = full_data[
        ['uid', 'participant_id', 'video_id', 'description', 'verb_class']]
    
    final_annotations.to_pickle(f"an_multimodal_annotations/S04_{split}.pkl")   
    
def create_emg_features(full_data, split, spectogram=False):
    
    full_data = pd.read_pickle(full_data)
    full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'emg_data': 'features_EMG'})
    
    emg_features = full_data[
        ['uid', 'features_EMG']]
    emg_features = emg_features.to_dict('list')
    emg_features = {'features': emg_features}

    features_name = f'saved_features_an_multimodal/features_emg_spectogram_S04_{split}.pkl' if spectogram \
        else f'saved_features_an_multimodal/features_emg_S04_{split}.pkl'
    
    with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)

In [82]:
train_df = take_S04_annotations_RGB(timestamps='S04_1.pkl', emg_data='new_emg_data_test.pkl', type='test')
test_df = take_S04_annotations_RGB(timestamps='S04_1.pkl', emg_data='new_emg_data_train.pkl', type='train')
create_reduced_annotations(train_annotations=train_df, test_annotations=test_df)

In [88]:
create_multimodal_annotations(full_data="new_emg_data_train.pkl", split='train')
create_multimodal_annotations(full_data="new_emg_data_test.pkl", split='test')
create_emg_features(full_data="new_emg_data_train.pkl", split='train')
create_emg_features(full_data="new_emg_data_test.pkl", split='test')

In [47]:
data2 = pd.read_pickle('new_emg_data_test.pkl')

data2

,index,file,description,labels,start,stop,emg_data,description_class,uid
0,11,S03_1.pkl,Get items from refrigerator/cabinets/drawers,Get/Put,1.655228e+09,1.655228e+09,"[[-0.5940820467598136, -0.2558365791189061, -0...",0,120
1,16,S07_1.pkl,Clear cutting board,Clear,1.657726e+09,1.657726e+09,"[[-0.9999970501763059, -0.9999994636684193, -0...",18,608
2,10,S07_1.pkl,Slice a cucumber,Slice,1.657726e+09,1.657726e+09,"[[-0.3977339667409513, -0.5612539489833003, -0...",2,565
3,4,S08_1.pkl,Peel a cucumber,Peel,1.657737e+09,1.657737e+09,"[[-0.701509930321272, -0.5582205855583406, -0....",1,728
4,40,S09_2.pkl,Spread jelly on a bread slice,Spread,1.657809e+09,1.657809e+09,"[[-0.7193676857524235, -0.6259481828826229, -0...",7,892
...,...,...,...,...,...,...,...,...,...
897,34,S05_2.pkl,Open a jar of almond butter,Open/Close,1.655256e+09,1.655256e+09,"[[-0.31753439615059575, -0.8357486489010604, -...",8,348
898,54,S08_1.pkl,Get items from refrigerator/cabinets/drawers,Get/Put,1.657739e+09,1.657739e+09,"[[-0.8650372466946141, -0.8247769717071822, -0...",0,820
899,17,S05_2.pkl,Slice a potato,Slice,1.655255e+09,1.655255e+09,"[[-0.7490247862263646, -0.6959249467341944, -0...",4,307
900,20,S07_1.pkl,Slice a potato,Slice,1.657727e+09,1.657727e+09,"[[0.05507003809854874, -0.023589837248776467, ...",4,642


In [62]:
data2 = pd.read_pickle('action_net_annotations_training/S04_test.pkl')

data2

,index,uid,participant_id,video_id,narration,start,stop,start_frame,stop_frame,verb,verb_class
2,27,2,S04,S04_1,Open a jar of almond butter,1.655241e+09,1.655241e+09,69557,69855,Open a jar of almond butter,8
6,45,6,S04,S04_1,Clean a plate with a towel,1.655242e+09,1.655242e+09,79553,79751,Clean a plate with a towel,11
8,65,8,S04,S04_1,Open a jar of almond butter,1.655241e+09,1.655241e+09,69449,69747,Open a jar of almond butter,8
22,42,22,S04,S04_1,Clear cutting board,1.655241e+09,1.655241e+09,60141,60437,Clear cutting board,18
57,9,57,S04,S04_1,Clean a pan with a towel,1.655242e+09,1.655242e+09,81001,81180,Clean a pan with a towel,13
...,...,...,...,...,...,...,...,...,...,...,...
739,41,739,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",1.655242e+09,1.655242e+09,88632,88929,"Set table: 3 each large/small plates, bowls, m...",15
771,21,771,S04,S04_1,Clear cutting board,1.655241e+09,1.655241e+09,60443,60740,Clear cutting board,18
779,48,779,S04,S04_1,Spread almond butter on a bread slice,1.655241e+09,1.655241e+09,64415,64713,Spread almond butter on a bread slice,6
781,13,781,S04,S04_1,Spread almond butter on a bread slice,1.655241e+09,1.655241e+09,64379,64677,Spread almond butter on a bread slice,6


In [72]:
data2 = pd.read_pickle('action_net_annotations_training/S04_train.pkl')

data2.iloc[702]

index                           660
uid                             782
participant_id                  S04
video_id                      S04_1
narration               Slice bread
start             1655241083.635779
stop              1655241093.525394
start_frame                   59083
stop_frame                    59380
verb                    Slice bread
verb_class                        5
Name: 782, dtype: object

In [76]:
data2 = pd.read_pickle('saved_features_action_net/features_action_net_25_dense_S04_test.pkl')

data2 = pd.DataFrame.from_dict(data2['features'])
data2

,uid,video_name,features_RGB
0,2,S04_1,"[[0.20252235, 0.12722212, 0.41020927, 0.097577..."
1,6,S04_1,"[[0.29197595, 0.08010393, 0.37297875, 0.156138..."
2,8,S04_1,"[[0.12584968, 0.19852735, 0.494157, 0.06268781..."
3,22,S04_1,"[[0.21563528, 0.07804283, 0.52215993, 0.057511..."
4,57,S04_1,"[[0.26691076, 0.09130219, 0.2932961, 0.0329155..."
...,...,...,...
76,739,S04_1,"[[0.053627267, 0.2878403, 0.4117053, 0.0260446..."
77,771,S04_1,"[[0.20735748, 0.04445891, 1.1320182, 0.0788918..."
78,779,S04_1,"[[0.07726825, 0.1913928, 0.54565656, 0.3260083..."
79,781,S04_1,"[[0.13755912, 0.11520604, 0.4385789, 0.2291081..."


In [89]:
data2 = pd.read_pickle('saved_features_an_multimodal/features_emg_S04_test.pkl')

data2 = pd.DataFrame.from_dict(data2['features'])
data2

,uid,features_EMG
0,249,"[[-0.5310362499607831, -0.17112101585733755, -..."
1,204,"[[-0.06765700081485183, -0.19264746771383182, ..."
2,266,"[[-0.8781975985563716, -0.8737516029999017, -0..."
3,224,"[[-0.5560823621930757, -0.3788598220978606, -0..."
4,237,"[[-0.6652617227946247, -0.35463073586700183, -..."
...,...,...
76,268,"[[-0.8913982997448366, -0.8646481789321555, -0..."
77,248,"[[-0.32349131425931404, -0.19182720534648318, ..."
78,245,"[[-0.5547529179641489, -0.3992432882881073, -0..."
79,220,"[[-0.3703662356774267, -0.49132828506191123, -..."


In [90]:
data2 = pd.read_pickle('an_multimodal_annotations/S04_test.pkl')

data2

,uid,participant_id,video_id,description,verb_class
14,249,S04,S04_1,Open a jar of almond butter,8
22,204,S04,S04_1,Clear cutting board,18
34,266,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",15
37,224,S04,S04_1,Spread almond butter on a bread slice,6
39,237,S04,S04_1,Open a jar of almond butter,8
...,...,...,...,...,...
850,268,S04,S04_1,"Set table: 3 each large/small plates, bowls, m...",15
855,248,S04,S04_1,Open a jar of almond butter,8
880,245,S04,S04_1,Open a jar of almond butter,8
889,220,S04,S04_1,Spread almond butter on a bread slice,6
